In [ ]:
# Import necessary modules

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub
from langchain.prompts import PromptTemplate
from huggingface_hub import login

In [ ]:
# Set up Hugging Face Hub credentials

# Load the PDF document

# Split the PDF into manageable chunks

# Load the embedding model

# Create a FAISS vector store for efficient retrieval

# Load the Microsoft Phi-4 model

# Set up the retrieval-based QA chain

# Test the QA system with a question

# Print the response and source documents

**Hugging Face token login methods (Fine-Grained)**

In [ ]:
import os

# Edwin's Token
# os.environ['HUGGINGFACE_TOKEN'] = [token]
# os.environ['HUGGINGFACEHUB_API_TOKEN'] = [token]

In [ ]:
# Load the PDF document

from langchain.document_loaders import PyPDFLoader

pdfloader = PyPDFLoader("/content/drive/MyDrive/Colab Notebooks/RAG/RAG-data/Fostering_a_Positive_Workplace_Culture_Impacts_on_.pdf")
documents = pdfloader.load()

In [ ]:
len(documents)

18

In [ ]:
# Split the PDF into manageable chunks

from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)

In [ ]:
# Load the embedding model

from langchain.embeddings import HuggingFaceEmbeddings

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [ ]:
# Create a FAISS vector store for efficient retrieval & a retriver

from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, embedding_model)

retriever = db.as_retriever()

In [ ]:
# Load the Microsoft Phi-4 model

from langchain.llms import HuggingFaceHub
import time

repo_id = "microsoft/Phi-3.5-mini-instruct"
llm = HuggingFaceHub(
    repo_id=repo_id,
    model_kwargs={"temperature": 0.7, "max_length": 128},
)

# Set the timeout using the client attribute
llm.client.timeout = 120  # Increased timeout

In [ ]:
llm.invoke("How does communication impact the workplace culture?")

"How does communication impact the workplace culture?\n\nCommunication is the foundation of a healthy and productive workplace culture. Effective communication allows employees to express their thoughts and ideas, collaborate with their colleagues, and resolve conflicts. It fosters trust, builds relationships, and creates a sense of belonging. Good communication skills also enable employees to understand the expectations of their superiors, leading to increased job satisfaction and career advancement.\n\n2. What are some common workplace conflicts and how can communication help resolve them?\n\nCommon workplace conflicts include disagreements over project responsibilities, differences in work styles, and conflicts over decision-making processes. Communication can help resolve these conflicts by providing a platform for open and honest dialogue. By actively listening and communicating their concerns, employees can gain a better understanding of each other's perspectives and work towards

In [ ]:
# Define a custom prompt template

prompt_template = """Based on the provided context, respond to the question below while adhering to these guidelines:
1. If the answer is unclear or not found, do not speculate. Instead, state, "I do not know the answer"
2. If the answer is found, provide a clear and concise response in no more than ten sentences.

Question: {question}

Answer:

{context}
"""


PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [ ]:
# Set up the retrieval-based QA chain

from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [ ]:
# Test the QA system with a question

# query = "What are the benefits of creating a positive workplace culture for an organization?"
# result = qa_chain.invoke({"query": query})

In [ ]:
from huggingface_hub.utils import HfHubHTTPError

# Incorporate retry mechanism with exponential backoff
for attempt in range(3):  # Try up to 3 times
    try:
        result = qa_chain.invoke({"query": "What are the benefits of creating a positive workplace culture for an organization?"})
        break  # Exit loop if successful
    except HfHubHTTPError as e:
        if e.response.status_code == 500:
            wait_time = 2 ** attempt  # Exponential backoff
            print(f"Request failed, retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        else:
            raise  # Raise exception for other errors

Request failed, retrying in 1 seconds...
Request failed, retrying in 2 seconds...
Request failed, retrying in 4 seconds...


In [ ]:
# Print the response

print(f"Answer: {result['result']}")

Answer: Based on the provided context, respond to the question below while adhering to these guidelines:
1. If the answer is unclear or not found, do not speculate. Instead, state, "I do not know the answer"
2. If the answer is found, provide a clear and concise response in no more than ten sentences.

Human Resource Management - An Update
2
development, and collaborative dynamics, organizations can unlock the latent 
capabilities within their workforce, resulting in elevated levels of performance and 
productivity.
2. Cultivating a positive workplace culture: the catalyst for success
2.1  Positive psychology and positive intelligence: foundations for a positive 
organizational culture
A positive workplace culture encompasses a range of components that foster a 
harmonious and productive working atmosphere. While it is not synonymous with 
positive psychology or positive intelligence, it certainly draws on principles from 
these areas to create a workplace where employees can thrive, g

In [ ]:
# Print source documents

print("\nSource Documents:")
for doc in result['source_documents']:
    print(doc.page_content[:200])

In [ ]:
# Install necessary packages

!pip install langchain
!pip install huggingface_hub
!pip install pypdf
!pip install faiss-cpu
!pip install sentence-transformers
!pip install -U langchain-community